# Lesson 2 - Assignment

In this assignment, you will implement a Decision Tree algorithm from scratch and compare the results to existing sklearn algorithm. 

In [14]:
# import packages
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from matplotlib.legend_handler import HandlerLine2D
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

# make this notebook's output stable across runs
np.random.seed(0)

Question 1.1: Implement the functions to calculate Gini Index.

In [15]:
# Calculate the Gini index for a split dataset
def gini_index(groups, classes):
    # count all samples at split point
    n_instances = float(sum([len(group) for group in groups]))
    # sum weighted Gini index for each group
    gini = 0.0
    for group in groups:
        size = float(len(group))
        # avoid divide by zero
        if size == 0:
            continue
        score = 0.0
        # score the group based on the score for each class
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val) / size
            score += p * p
        # weight the group score by its relative size
        gini += (1.0 - score) * (size / n_instances)
    return gini

Question 2.1: Write a method that splits the dataset based on an atrribute and an attribute value

In [16]:
# Split a dataset based on an attribute and an attribute value

def test_split(index, value, dataset):
    left, right = list(), list()
    """
    TODO: This function loops over each row and checks if the row belongs to the right or left list.
    """ 
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right

Question 2.2: Write a method that loops over the dataset, determine the groups of rows that belong to the right or left split, and calculates the gini_index

In [17]:
def get_split(dataset):
    "TODO Select the best split point for a dataset"
    class_values = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    for index in range(len(dataset[0])-1):
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            gini = gini_index(groups, class_values)
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    return {'index':b_index, 'value':b_value, 'groups':b_groups}

Question 2.3: Repeat question 2.2 using entropy

In [18]:
def get_split_entropy(dataset):
    "TODO Select the best split point for a dataset"
    class_values = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    for index in range(len(dataset[0])-1):
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            entropy = entropy(groups, class_values)
            if entropy < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], entropy, groups
    return {'index':b_index, 'value':b_value, 'groups':b_groups}

# implement an entropy function based on the formula  
# Entropy = -SUM(p(x) * log2 (p(X)))   where p(X) = #x/n
def entropy(groups, classes):
    total = class1 + class2
    p1 = class1 / total
    p2 = class2 / total
    entropy = -p1 * math.log2(p1) - p2 * math.log2(p2)
    return entropy

Question 3.1: Write a method that takes in a group of rows and determines the class they belongs to. It should return the most common output value for a list of rows.

In [19]:
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)

Question 3.2: Write a method that recursively split the data.
The method takes in a node, max_depth, min_size, and depth. Initially, the method would be called by passing the rood node and depth of 1. Here are the steps to help you implement:

- First, we create two groups for the data split and delete any existing groups from the node. As rows are used, they are no longer needed.
- Second, check if rows should be in left or right group, and if so we create a terminal node using the records we have.
- Third, check if maximum depth is reached and if so we create a terminal node.
- Fourth, process left child, creating a terminal node if the group of rows is too small, otherwise creating and adding the left node in a depth first fashion until the bottom of the tree is reached on this branch.
- Fifth, process the right side in a similar manner as left side, as we rise back up the constructed tree to the root.

In [20]:
def split(node, max_depth, min_size, depth):
    left, right = node['groups']
    del(node['groups'])
    # check for a no split
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    # check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    # process left child
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left)
        split(node['left'], max_depth, min_size, depth+1)
    # process right child
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right)
        split(node['right'], max_depth, min_size, depth+1)

Question 3.3: Write a method that builds the tree. The method creates an initial split to determine root node, and then calls the split method.

In [21]:
# Build a decision tree
def build_tree(train, max_depth, min_size):
    root = get_split(train)
    split(root, max_depth, min_size, 1)
    return root

Question 3.4: Write a method that takes in a node and rows of data, and predicts the class associated with each row.

In [22]:
# Make a prediction with a decision tree
def predict(node, row):
    "TODO check if a row belongs to a node and recursively traverse the tree if the row doesn't."
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']

Question 4: Train a decision tree using the banknote_authentication data

In [23]:
from random import seed
from random import randrange
from csv import reader
from sklearn.metrics import accuracy_score

# Load a CSV file
def load_csv(filename):
    file = open(filename, "rt")
    lines = reader(file)
    dataset = list(lines)
    return dataset
 
# Convert string column to float
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())

# Convert string column to int
# This didn't work. So I used the function str_column_to_float instead.
def str_column_to_int(dataset, column):
    for row in dataset:
        row[column] = int(row[column].strip())
        
filename = 'data_banknote_authentication.csv'
dataset = load_csv(filename)
# convert string attributes to integers
for i in range(len(dataset[0])):
    str_column_to_float(dataset, i) # I tried using the function str_column_to_int, but it didn't work.  

train = dataset[1:int(len(dataset)*2/3)]
test = dataset[int(len(dataset)*2/3)+1:len(dataset)]

In [24]:
#Build a tree and evalute accuracy
tree = build_tree(train, 10, 1)
predictions = list()
for row in test:
    prediction = predict(tree, row)
    predictions.append(prediction)
               
print('Accuracy: %s' % accuracy_score([row[-1] for row in test], predictions))

Accuracy: 0.9671772428884027


[Bonus] Question 5: Train and evaluate an SKLEARN decision tree model, and compare the results to your model 

In [26]:
import numpy as np
# Train and evaluate a Sklearn decisio tree model and compare accuracy with the implemented model
X_train = np.array([row[:-1] for row in train])
y_train = np.array([row[-1] for row in train])
X_test = np.array([row[:-1] for row in test])
y_test = np.array([row[-1] for row in test])

# Train a decision tree model
tree = DecisionTreeClassifier(criterion='gini', max_depth=10, min_samples_split=2, min_samples_leaf=1, random_state=42)
tree.fit(X_train, y_train)

# Make predictions
y_pred = tree.predict(X_test)

# Evaluate the model
print('Accuracy: %s' % accuracy_score(y_test, y_pred))

Accuracy: 0.949671772428884


Question 6: Create a new text cell in your Notebook: Complete a 50-100 word summary (or short description of your thinking in applying this week's learning to the solution) of your experience in this assignment. Include: What was your incoming experience with this model, if any? what steps you took, what obstacles you encountered. how you link this exercise to real-world, machine learning problem-solving. (What steps were missing? What else do you need to learn?) This summary allows your instructor to know how you are doing and allot points for your effort in thinking and planning, and making connections to real-world work.

This week's assignment was a very interesting exercise to build our own decision tree from scratch. In fact, I realized about half-way through the exercise I remembered that I had already built a decision tree from scratch using a [tutorial from AssemblyAI](https://www.youtube.com/watch?v=NxEHSAfFlK8). However looking at the code that I copied from AssemblyAI and the code from this exercise, it is quite different. The most obvious is that we are using the Gini Index to decide on splits and the example from AssemblyAI uses Entropy metric to make the same splits. 

With regards to the results I obtained. I had the following accuracy on the given banknote_authentication.csv dataset with my "from scratch" decision tree impleentation:
``` Accuracy: 0.9671772428884027 ```

And using the Sklearn libraries I obtained the following accuracy:
```Accuracy: 0.949671772428884```

It is notable that the accuracy achieved with the "from scratch" implementation was slightly higher (~2%) that that achieved from the Sklearn libraries. I tried to keep as many parameters as possible the same (criterion, max_depth, min_size) I also did not attempt any hyper-parameter tuning to improve performance using grid search or anything like that.